In [2]:
import tensorflow as tf
import numpy as np

import warnings
## Tensorflow produces a lot of warnings. We generally want to suppress them. The below code does exactly that. 
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Using predefined estimators
In this notebook we recreate the neural network defined first in notebook [3.Neural-Networks.ipynb](3.Neural-Networks.ipynb)  
using the predefined estimator `DNNClassifier`

## Benefits of Estimators

- Estimator-based models are independent of operating environment
    - local host
    - GPUs
    - CPU clusters

### More benefits
- Simplify model sharing between developers
- State of the art model architectures with more intuitive high-level code

Consult https://www.tensorflow.org/programmers_guide/estimators for more advantages of using Estimators as described by the developers of TensorFlow.

## Read Data
The MNist dataset is available from within TensorFlow tutorials.

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

# Wrap input as a function (THE "input function" will be defined below)
def input(dataset):
    return dataset.images, dataset.labels.astype(np.int32)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


##  Define feature columns

In [5]:
# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

## Define Neural Network

In [13]:
# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 256],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir="./tmp/mnist_model_256_256"   # Location for storing checkpoints.
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/mnist_model_256_256', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f00735cb940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Define training input function
* Supplies data for training, evaluation, prediction
* Should yield tuples of:
    - Python dict `features`: key = name of feature, value = array of feature values
    - Array `label` : label for every example

In [20]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

## Train the neural network
* Checkpoint used for "warm start"
* Checkpoints saved

In [21]:
tf.logging.set_verbosity(tf.logging.INFO)
classifier.train(input_fn=train_input_fn, steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model_256_256/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into ./tmp/mnist_model_256_256/model.ckpt.
INFO:tensorflow:loss = 10.7845, step = 1001
INFO:tensorflow:global_step/sec: 285.869
INFO:tensorflow:loss = 20.021835, step = 1101 (0.351 sec)
INFO:tensorflow:global_step/sec: 352.784
INFO:tensorflow:loss = 7.6053376, step = 1201 (0.283 sec)
INFO:tensorflow:global_step/sec: 334.544
INFO:tensorflow:loss = 12.817944, step = 1301 (0.299 sec)
INFO:tensorflow:global_step/sec: 344.687
INFO:tensorflow:loss = 21.905014, step = 1401 (0.290 sec)
INFO:tensorflow:global_step/sec: 340.552
INFO:tensorflow:loss = 7.4117017, step = 1501 (0.294 sec)
INFO:tensorflow:global_step/sec: 334.818
INFO:tensorf

INFO:tensorflow:global_step/sec: 349.703
INFO:tensorflow:loss = 6.8408923, step = 9001 (0.286 sec)
INFO:tensorflow:global_step/sec: 356.258
INFO:tensorflow:loss = 3.9635491, step = 9101 (0.281 sec)
INFO:tensorflow:global_step/sec: 366.011
INFO:tensorflow:loss = 1.77024, step = 9201 (0.273 sec)
INFO:tensorflow:global_step/sec: 369.635
INFO:tensorflow:loss = 8.391723, step = 9301 (0.271 sec)
INFO:tensorflow:global_step/sec: 362.501
INFO:tensorflow:loss = 0.45997304, step = 9401 (0.276 sec)
INFO:tensorflow:global_step/sec: 368.357
INFO:tensorflow:loss = 1.7720134, step = 9501 (0.271 sec)
INFO:tensorflow:global_step/sec: 369.648
INFO:tensorflow:loss = 5.812295, step = 9601 (0.271 sec)
INFO:tensorflow:global_step/sec: 366.741
INFO:tensorflow:loss = 4.0835114, step = 9701 (0.271 sec)
INFO:tensorflow:global_step/sec: 369.925
INFO:tensorflow:loss = 6.994205, step = 9801 (0.271 sec)
INFO:tensorflow:global_step/sec: 364.982
INFO:tensorflow:loss = 5.3393254, step = 9901 (0.274 sec)
INFO:tensorflo

In [11]:
# Have a look at the checkpoint directory.
!ls -lrt ./tmp/mnist_model_256_256/ | tail

-rw-r--r-- 1 jovyan users  138840 Jun  5 22:14 model.ckpt-11001.meta
-rw-r--r-- 1 jovyan users     808 Jun  5 22:14 model.ckpt-11001.index
-rw-r--r-- 1 jovyan users 3231880 Jun  5 22:14 model.ckpt-11001.data-00000-of-00001
-rw-r--r-- 1 jovyan users  313784 Jun  5 22:14 graph.pbtxt
-rw-r--r-- 1 jovyan users  138840 Jun  5 22:15 model.ckpt-21000.meta
-rw-r--r-- 1 jovyan users     808 Jun  5 22:15 model.ckpt-21000.index
-rw-r--r-- 1 jovyan users 3231880 Jun  5 22:15 model.ckpt-21000.data-00000-of-00001
-rw-r--r-- 1 jovyan users 1017534 Jun  5 22:15 events.out.tfevents.1528236907.5538709837a2
-rw-r--r-- 1 jovyan users     275 Jun  5 22:15 checkpoint
drwxr-xr-x 4 jovyan users     128 Jun  5 22:15 eval


## Define test input function

In [23]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False
)

## Evaluate accuracy

In [24]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-05-19:53:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model_256_256/model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-05-19:53:15
INFO:tensorflow:Saving dict for global step 11000: accuracy = 0.9765, average_loss = 0.07637801, global_step = 11000, loss = 9.668102

Test Accuracy: 97.649997%

